In [1]:
# Transport for Toronto API Example
import requests

url = "https://myttc.ca/developers"
headers = {
    "Content-Type": "application/json"
}

response = requests.get(url)
# data = response.json()
print(response.status_code)

502
